# Data Preprocessing

- Build a data processing pipeline that takes the raw data and transforms them into a NN trainable dataset

* Transform Data from row values to coulmn based dataframe
* Clean Column names
* ...

## Imports

In [5]:
# data processing
import numpy as np
import pandas as pd

# data viz
import matplotlib.pyplot as plt
import seaborn as sn 

# gui
from tqdm.auto import tqdm
tqdm.pandas()


#funtions

def doing(string):
    print(string+"...", end="")

def done():
    print("✅")
    #print(" \033[91m\033[1mDone.\033[0m")
    
def choose_target_period(patition,target_name="ICP_Vital", mins_before_first_icp = 1440, min_icp_val=-10, max_icp_val=105):
    '''
    1. Filter ICP outliers taht are outside the 0.999 quantile (-10,105)
    2. Drop steps after last ICP measurement
    3. Shift relative time to start with 0 after removing rows
    
        Parameters:
                patition (DataFrame): The Dataframe we want to transform
                target_name(str): The column name of the ICP measurement value
                mins_before_first_icp (int): The number of minutes before the first ICP we want to capture
                min_icp_val (int): The minimum icp value that will be kept in the dataframe
                max_icp_val (int): The maximum icp value that will be kept in the dataframe                
                
        Return:
            patition (DataFrame): The transformed dataframe that was passed to the function
    
    '''
    # DF with only rows that have ICP as "Maßnahme"to check when the first and last ICP happened
    target_rows = patition[patition["Maßnahme"] == target_name]
    
    # create a mask that is False where values are outside specified range
    outlier_mask = (target_rows["Wert"] >= min_icp_val) & (target_rows["Wert"] <= min_icp_val)
    target_rows = target_rows[outlier_mask]
    
    
    # Check is patition has steps
    if len(target_rows) == 0:
        return patition.iloc[0:0]
    
    # kick out targets beyond thresholds
    min_time_threashold = min(target_rows["rel_time"]) # since target steps only has ICP measurements, this is the first ICP measurement for the patient
    min_time_threashold = min_time_threashold - mins_before_first_icp # But we want to have 1440 minutes before the first measurement, so we need to substract it
    max_time_threashold = max(target_rows["rel_time"]) # We want to remove rows that occure after the last ICP
    
    # kick out (basically set to NaN) measurements beyond interesting region
    patition = patition[patition["rel_time"] >= min_time_threashold]
    patition = patition[patition["rel_time"] <= max_time_threashold]
    
    # subtract the minimum time value from al values, to shift the first row to have a rel_time of 0
    patition["rel_time"] -= min(patition["rel_time"])


    return patition

def rename_nbd(name):
    '''
    merge "NBD" (non-invasive ways of measuring blood pressure) with invasive ways as they measure the same thing
    
        Parameters:
            name(string): the name of the measurement we want to rename
        Return:
            name(string): the name after we have altered it to the generel name
    '''
    if "syst" in name:
        name = "syst_Vital"
    elif "diast" in name:
        name = "diast_Vital"
    elif "mittl" in name:
        name = "mittl_Vital"
        
    return name

# Load the dataset 
doing("Loading Dataset")
datenbank_werte_df = pd.read_csv("../icp_prediction/data/Datenbank_Werte.csv")
done()## Create ICP data set 
Can be skipped if file already exists.

In [7]:
# Load the dataset 
doing("Loading Dataset")
datenbank_werte_df = pd.read_csv("../master-thesis/data/Datenbank_Werte.csv")
done()

Loading Dataset...

/tmp/ipykernel_800928/576936030.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  datenbank_werte_df = pd.read_csv("../master-thesis/data/Datenbank_Werte.csv")


✅


In [8]:
datenbank_werte_df.head(3)

,Pat_ID,ID,Maßnahme,Maßnahme_norm,rel_time,Wert,DB
0,123456798,BGA,Glu,Glu,-2880.0,100.00,UKE
1,474097135,BGA,Ca,Ca,419.0,1.23,UKE
2,474097135,BGA,Ca,Ca,492.0,1.28,UKE


In [64]:
# Loade the dataset 
doing("Loading Dataset")
datenbank_werte_df = pd.read_csv("../icp_prediction/data/Datenbank_Werte.csv")
done()

# remove the eICU values from the dataset
doing("Remove eICU values")
datenbank_werte_df = datenbank_werte_df[datenbank_werte_df["DB"] != "eICU"]
done()

# add the type of the Masßnahme (ID = ['BGA' 'Labor' 'Vital' 'Med']) to the Masßnahme value and drop the ID column
doing("Combine Maßnahme and ID column")
datenbank_werte_df["Maßnahme"] += "_" + datenbank_werte_df["ID"]
datenbank_werte_df = datenbank_werte_df.drop(columns="ID")
done()

# drop measurements where we have a "NAN"
nan_values = datenbank_werte_df['Wert'].isnull().sum()
doing("Dropping {} nan values".format(nan_values))
datenbank_werte_df = datenbank_werte_df.dropna(subset=["Wert"])
done()

# drop duplicates
doing("Dropping duplicates")
datenbank_werte_df = datenbank_werte_df.drop_duplicates(subset=["Pat_ID", "Maßnahme", "rel_time", "Wert"])
done()

# groupby the Pat ID and apply choose_target_period to each patition
doing("Remove outliers and shift relative time to start with 0")
datenbank_werte_df = datenbank_werte_df.groupby("Pat_ID").apply(lambda patition: choose_target_period(patition))
done()

# resetting the DF index, becasue we have removed many rows
doing("Reset index")
datenbank_werte_df = datenbank_werte_df.reset_index(drop=True)
done()

# renaming NBDs because multiple measures measure the same thing
doing("Renaming NBDs")
datenbank_werte_df["Maßnahme"] = datenbank_werte_df["Maßnahme"].apply(rename_nbd)
done()

# save the DF to a .csv file
file_name = "data/Datenbank_icp.csv"
doing("Saving dataframe to '{}'".format(file_name))
datenbank_werte_df.to_csv(file_name, index=False)
done()

Loading Dataset...✅
Remove eICU values...✅
Combine Maßnahme and ID column...✅
Dropping 4271 nan values...✅
Dropping duplicates...✅
Remove outliers and shift relative time to start with 0...✅
Reset index...✅
Renaming NBDs...✅
Saving dataframe to 'data/Datenbank_icp.csv'...✅


## Loade the 'Datenbank_icp.csv' file
- determine columns that have few missings
- select high count vitals 

In [66]:
# load the 'Datenbank_icp.csv' file
icp_df = pd.read_csv("data/Datenbank_icp.csv")

In [9]:
icp_df_anton = pd.read_csv("../icp_prediction/data/Datenbank_icp.csv")

/tmp/ipykernel_800928/2177001019.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  icp_df_anton = pd.read_csv("../icp_prediction/data/Datenbank_icp.csv")


In [10]:
icp_df_anton.head()

,Pat_ID,Maßnahme,Maßnahme_norm,rel_time,Wert,DB
0,141800,Ca_BGA,Ca,540.0,8.5,eICU
1,141800,Ca_BGA,Ca,1060.0,8.0,eICU
2,141800,Cl_BGA,Cl,540.0,107.0,eICU
3,141800,Cl_BGA,Cl,1060.0,115.0,eICU
4,141800,FiO2_BGA,FO2,74.0,100.0,eICU


In [67]:
print(len(icp_df["Pat_ID"]))
print(len(icp_df_anton["Pat_ID"]))

612768
81746137
